# グラフアルゴリズム1 (最短経路問題)
最短経路問題の種類
* 2頂点対最短経路問題ー特定の2つのノード間の最短距離を求める
* 単一始点最短経路問題ーある始点ノードからほかの全部のノードへの最短経路を求める
* 全点対最短経路問題ーすべての2ノード間の最短経路を求める


# ダイクストラ法を用いた2頂点間の最短経路問題

距離がバラバラだとBFSは使えない。→重要な性質「最短経路の部分経路も最短経路」

## ダイクストラ法の手順

1. **初期化**:
    - 各ノードの最短距離を、十分に大きい数字で初期化。
    - 開始ノードの距離を0に設定。

2. **開始ノードの処理**:
    - 開始ノードからスタートし、直接繋がっているノードに対して接続する辺の距離を参照し、そのノードの現時点での最短距離を記録。

3. **確定済みノードの選定**:
    - 開始ノードの処理が終わったら、そのノードを確定済みとする。
    - 次に、最短距離が初期化されたときと違う値になっていて、かつ最短距離が確定されていないノードのうち、現時点で最短距離が最も小さいものを選び出す。

4. **距離の更新**:
    - 直接繋がっているノードに対して接続する辺の距離を参照し、その辺を使うことでそのノードの現時点での最短距離を更新できる場合は、更新。

5. **確定**:
    - このノードを確定とする。
    - 以降、全てのノードが確定するまで #3 と #4 を繰り返す。

In [1]:
edges_list = [[[1, 5], [2, 4]], [[0, 5], [3, 3], [5, 9]], [[0, 4], [3, 2], [4, 3]], [[1, 3], [2, 2], [5, 1], [6, 7]], [[2, 3], [6, 8]], [[1, 9], [3, 1], [6, 2], [7, 5]], [[3, 7], [4, 8], [5, 2], [7, 2]], [[5, 5], [6, 2]]]

def dijkstra(V, e_list): 
    inf = 10**10
    done = [False]*V 
    dist = [inf]*V
    dist[0] = 0
    while 1:
        tmp_min_dist=inf
        cur_node=-1
        for i in range(V):
            if (not done[i]) and (tmp_min_dist>dist[i]):
                tmp_min_dist=dist[i]
                cur_node=i
        if cur_node==-1:
            break
        for e in e_list[cur_node]:
            if dist[e[0]]>dist[cur_node]+e[1]:
                dist[e[0]]=dist[cur_node]+e[1]
        done[cur_node]=True
    print(dist)

dijkstra(8,edges_list)

[0, 5, 4, 6, 7, 7, 9, 11]


計算量はwhileループがすべてのノードを見るまで回り続けるためO(V)  
最初のforループは常にO(|V|)よりwhileループも併せて$O(|V|^2)$となる。  
##### 改善→ヒープの使用
1. **ヒープから，現在までで最短距離を持つ未確定のノードを取り出す（ヒープが更新される）**
2. **distを更新する．distの更新があればヒープの更新（要素の追加）を⾏う**
3. **全ノード終わるまで，#1に戻る**

In [2]:
import heapq

def dijkstra_heap(V,e_list):
    inf=10**10
    done=[False]*V
    dist=[inf]*V
    dist[0]=0
    node_heap=[]
    heapq.heappush(node_heap,[dist[0],0])
    #ヒープの要素がなくなるまで
    while node_heap:
        tmp=heapq.heappop(node_heap)
        cur_node=tmp[1]
        if not done[cur_node]: #未訪問
            for e in e_list[cur_node]:
                if dist[e[0]]>dist[cur_node]+e[1]:
                    dist[e[0]]=dist[cur_node]+e[1]
                    #更新時にヒープに入れる
                    heapq.heappush(node_heap,[dist[e[0]],e[0]])
        done[cur_node]=True
    print(dist)
            
dijkstra_heap(8,edges_list)

[0, 5, 4, 6, 7, 7, 9, 11]


ヒープに入る要素の数はO(|E|)よりdistの更新に伴う要素の追加も最短距離のノードを取り出すのも計算量はO(|E|log|E|)になる。 
ヒープの大きさはノードの数O(|V|)であるからdistの更新に伴う要素の更新はO(|E|log|V|) 
最短距離のノードを取り出すのにO(|V|log|V|)
したがって全体としてO((|V|+|E|)log|V|)となる 
#### さらに
フィボナッチヒープという特殊なヒープを使うとO(|E|+|V|log|V|)にできる


## ベルマン・フォード法
構造はダイクストラと同じだが、最短距離の選択を行わずに更新ごとにすべての辺に対しての計算を毎回行う  
負の経路があっても計算可能  
リストが始点、終点、距離の順であることに注意 

In [5]:
def BellmanFord(V,edges_list):
    inf=10**10
    dist=[inf]*V
    dist[0]=0
    for j in range(V):
        for e in edges_list: 
            if dist[e[1]] > e[2] + dist[e[0]]: 
                dist[e[1]] = e[2] + dist[e[0]] 
                # 負の閉路の検知
                if j==V-1: 
                    return -1  
    print(dist)
    
edges_list2 = [[0, 1, 5], [0, 2, 4], [1, 0, 5], [1, 3, 9], [1, 5, 9], [2, 0, 4], [2, 3, 2], [2, 4, 3], [3, 1, 9], [3, 2, 2], [3, 5, 1], [3, 6, 7], [4, 2, 3], [4, 6, 8], [5, 1, 9], [5, 3, 1], [5, 6, 2], [5, 7, 5], [6, 3, 7], [6, 4, 8], [6, 5, 2], [6, 7, 2], [7, 5, 5], [7, 6, 2]] 
BellmanFord(8, edges_list2)

[0, 5, 4, 6, 7, 7, 9, 11]


計算量は$ O(|V|^3) $

## SPFA
基本の考えはベルマン・フォードに同じだが毎回全部の辺をチェックすることを避けることで高速化を図る。  
ノードiのdist[i]に更新がなければ，そこから直接つながっているノードに対するdistの更新は必要ない  
つまり，dist[i]に更新が起きた時のみ，そこに接続するノードも更新する必要がある，として順次処理をする  
実装においては，更新が必要なノードが出てきたら，それをキューに⼊れる  
そのキューが空になるまでループを回す

In [6]:
from collections import deque
def spfa(V,e_list):
    inf=10**10
    dist=[inf]*V
    dist[0]=0
    # チェックが必要なノードを格納するキュー
    node_to_check = deque() 
    # キューに⼊っているかどうかのフラグ
    in_queue = [False]*V
    # 開始ノード（index：0）をキューに⼊れる
    node_to_check.append(0) 
    in_queue[0] = True 
    # 辺をチェックした数をカウント（本来は必要なし）
    count = 0
    # キューにノードがある限りループを回す．
    while node_to_check: 
        # キューから取り出し，このノードをチェック．
        cur_node = node_to_check.popleft() 
        in_queue[cur_node] = False
        # cur_nodeからつながっている辺をチェック．
        for e in e_list[cur_node]: 
            count += 1 
            if dist[e[0]] > dist[cur_node] + e[1]: 
                dist[e[0]] = dist[cur_node] + e[1]
                # 更新したらキューに⼊れる
                if not in_queue[e[0]]: 
                    in_queue[e[0]] = True 
                    node_to_check.append(e[0])
    print(dist) 
    print(count)

spfa(8, edges_list)

[0, 5, 4, 6, 7, 7, 9, 11]
24


計算量は$O(|V||E|)$だが、負の辺がないランダムケースでは実験的に$O(|E|)$になることが知られている。

## ワーシャルフロイド法  
2つのノードの全ての組み合わせに対して，最短経路（全点対最短経路）を導出するアルゴリズム  

In [ ]:
def WarshallFloyd(V, e_matrix): 
    # dist[i][j]：ノードiからノードjまで最短距離を保持する．
    # # 隣接⾏列を保持しておきたいならdeepcopyにする．
    inf=10**10
    dist = e_matrix
    for i in range(V):
        for j in range(V):
            for k in range(V):
                if dist[i][k]!=inf and dist[k][j]!=inf:
                    if dist[i][j]>dist[i][k]+dist[k][j]:
                        dist[i][j]=dist[i][k]+dist[k][j]
    print(dist)

In [ ]:
#課題10-a
import heapq
N,M,S=map(int,input().split())
List=[list(map(int,input().split())) for _ in range(M)]

def make_edges_list(L):
    edges_list=[[] for _ in range(N)]
    for u,v,w in L:
        edges_list[u].append((v, w))
    return edges_list

def dijkstra_heap(V,S,e_list):
    inf=10**30
    done=[False]*V
    dist=[inf]*V
    dist[S]=0
    node_heap=[]
    heapq.heappush(node_heap,[dist[0],S])
    #ヒープの要素がなくなるまで
    while node_heap:
        tmp=heapq.heappop(node_heap)
        cur_node=tmp[1]
        if not done[cur_node]: #未訪問
            for e in e_list[cur_node]:
                if dist[e[0]]>dist[cur_node]+e[1]:
                    dist[e[0]]=dist[cur_node]+e[1]
                    #更新時にヒープに入れる
                    heapq.heappush(node_heap,[dist[e[0]],e[0]])
        done[cur_node]=True
    return dist
            
edges_list=make_edges_list(List)            
dist=dijkstra_heap(N,S,edges_list)

for d in dist:
    if d==10**30:
        print('INF')
    else:
        print(d)

In [9]:
#課題10-b
inf=10**30

def make_adj_matrix(N,List):
    adj_matrix=[[inf]*N for _ in range(N)]
    for i in range(N):
        adj_matrix[i][i]=0
    for u,v,w in List:
        adj_matrix[u][v]=w 
    return adj_matrix

def WarshallFloyd(V,e_matrix): 
    dist=e_matrix    
    for k in range(V):
        for i in range(V):
            for j in range(V):
                if dist[i][k]!=inf and dist[k][j]!=inf:
                    if dist[i][j]>dist[i][k]+dist[k][j]:
                        dist[i][j]=dist[i][k]+dist[k][j]
    return dist

N,M=map(int, input().split())
List=[list(map(int, input().split())) for _ in range(M)]
Q=int(input())
query=[list(map(int,input().split())) for _ in range(Q)]

adj_matrix=make_adj_matrix(N,List)
dist_matrix=WarshallFloyd(N,adj_matrix)

for u,v in query:
    if dist_matrix[u][v]==inf:
        print("INF")
    else:
        print(dist_matrix[u][v])


ValueError: too many values to unpack (expected 2)

In [17]:
#Extra
import heapq
N,M,K=map(int,input().split())
List=[list(map(int,input().split())) for _ in range(M)]

def make_edges_list(L):
    edges_list=[[] for _ in range(N)]
    for u,v,w in L:
        edges_list[u-1].append((v-1,w))
        edges_list[v-1].append((u-1,w))
    return edges_list

def dijkstra_heap(V,S,e_list):
    inf=10**30
    days=[inf]*V
    dist=[inf]*V
    days[S]=0
    dist[S]=0
    node_heap=[]
    heapq.heappush(node_heap,[0,0,S])
    while node_heap:
        tmp=heapq.heappop(node_heap)
        cur_dist,cur_days,cur_node=tmp
        if cur_days>days[cur_node]:
            continue
        for neighbor,length in e_list[cur_node]:
            new_dist=cur_dist+length
            if new_dist<=K:
                if cur_days<days[neighbor] or cur_days==days[neighbor] and new_dist<dist[neighbor]:
                    days[neighbor]=cur_days
                    dist[neighbor]=new_dist
                    heapq.heappush(node_heap,(new_dist,cur_days,neighbor))
            else:
                if cur_days+1<days[neighbor] or (cur_days+1==days[neighbor] and length<dist[neighbor]):
                    days[neighbor]=cur_days+1
                    dist[neighbor]=length
                    heapq.heappush(node_heap,(length,cur_days+1,neighbor))
    
    return days
            
edges_list=make_edges_list(List)            
days=dijkstra_heap(N,0,edges_list)

for d in days[1:len(days)]:
    print(d+1)

1
2
2
1


: 

In [ ]:

5 4 4
1 5 3
3 4 1
2 1 3
2 3 3